In [2]:
# Mengimport Library
import pandas as pd

In [7]:
# Membaca Dataset
dataset_scoring_dqlab = pd.read_excel(r"credit_scoring_dqlab.xlsx")

print(dataset_scoring_dqlab.head())

   Unnamed: 0 kode_kontrak  pendapatan_setahun_juta kpr_aktif  \
0           0   AGR-000001                      295        YA   
1           1   AGR-000011                      271        YA   
2           2   AGR-000030                      159     TIDAK   
3           3   AGR-000043                      210        YA   
4           4   AGR-000049                      165     TIDAK   

   durasi_pinjaman_bulan  jumlah_tanggungan rata_rata_overdue  risk_rating  
0                     48                  5      61 - 90 days            4  
1                     36                  5      61 - 90 days            4  
2                     12                  0       0 - 30 days            1  
3                     12                  3      46 - 60 days            3  
4                     36                  0      31 - 45 days            2  


In [ ]:
# Data Preparation
